### Extracting Systematics using Geometric Subpixelisation at NSIDE = 2048



In [6]:
import time

import numpy as np
from astropy.io import fits
import healpy as hp
import pandas as pd
import matplotlib.path as mplPath
import matplotlib.pyplot as plt
import pickle
from collections import defaultdict

In [2]:
def raDec2thetaPhi(ra, dec):
    return (0.5 * np.pi - np.deg2rad(dec)), (np.deg2rad(ra))

NSIDE = 2048
NPIX = hp.nside2npix(NSIDE)

### Retrieving the relevant CCD Data

In [3]:
decamCCD = fits.open('../../../bricks_data/ccds-annotated-decam-dr9.fits')
mosaicCCD = fits.open('../../../bricks_data/ccds-annotated-mosaic-dr9.fits')
bassCCD = fits.open('../../../bricks_data/ccds-annotated-90prime-dr9.fits')
dataDecam = decamCCD[1].data
dataMosaic = mosaicCCD[1].data
dataBass = bassCCD[1].data


### Extracting CCD Informations from the entire Sample and Concatenating them together

In [4]:
#Extracting systematics
filter_colour = np.concatenate((dataDecam.field('filter'), dataMosaic.field('filter'), dataBass.field('filter')),
                               axis=0)
exptime = np.concatenate((dataDecam.field('exptime'), dataMosaic.field('exptime'), dataBass.field('exptime')), axis=0)
airmass = np.concatenate((dataDecam.field('airmass'), dataMosaic.field('airmass'), dataBass.field('airmass')), axis=0)
fwhm = np.concatenate((dataDecam.field('fwhm'), dataMosaic.field('fwhm'), dataBass.field('fwhm')), axis=0)
seeing = fwhm * 0.262
ccdskysb = np.concatenate((dataDecam.field('ccdskysb'), dataMosaic.field('ccdskysb'), dataBass.field('ccdskysb')),
                          axis=0)
meansky = np.concatenate((dataDecam.field('meansky'), dataMosaic.field('meansky'), dataBass.field('meansky')), axis=0)
galdepth = np.concatenate((dataDecam.field('galdepth'), dataMosaic.field('galdepth'), dataBass.field('galdepth')),
                          axis=0)
psfdepth = np.concatenate((dataDecam.field('psfdepth'), dataMosaic.field('psfdepth'), dataBass.field('psfdepth')),
                          axis=0)
psfnorm_mean = np.concatenate(
    (dataDecam.field('psfnorm_mean'), dataMosaic.field('psfnorm_mean'), dataBass.field('psfnorm_mean')), axis=0)
gausspsfdepth = np.concatenate(
    (dataDecam.field('gausspsfdepth'), dataMosaic.field('gausspsfdepth'), dataBass.field('gausspsfdepth')), axis=0)


### Now, trying to actually get all ccds per pixel
Begin by loading the Dictionary mapping pixels to CCDs. Afterwards, iterating through the ccds for a given pixel.
Depending on the systematics, we have to break systematics down into the different bands.
Airmass is simply averaged across all CCDs, exposure times are calculated on a per CCD basis.


### Import subpixel2ccd mapping


In [7]:
# Use this cell to simply import an existing pixel2ccd mapping

start = time.time()
# For DECAM, BASS, MzLS
with open('../../../bricks_data/pixel2ccd_2048_non_inclusive.pickle', 'rb') as f:
    pixel2ccd_dict = pickle.load(f)
    f.close()

print(time.time() - start /60)

1612141848.7527347


###  Get systematics value across every subpixel and average --> this is where the procedure differs from prev
- Iterate through all pixels
- Get systematic value at pixel
- average for pixel

### Adapt this loop to no longer iterate over subpixels

In [14]:
pixels_overall = pixel2ccd_dict.keys()
print(len(pixels_overall))

pixel2systematics_dict = defaultdict(list)

28536897


In [15]:
time_start = time.time()

# Find subpixels for the given pixel
for i, sample_pixel in enumerate(pixels_overall):
    airmass_aggregate = 0

    seeing_aggregate_g = 0
    seeing_aggregate_r = 0
    seeing_aggregate_z = 0

    ccdskysb_aggregate_g = 0
    ccdskysb_aggregate_r = 0
    ccdskysb_aggregate_z = 0

    exptime_aggregate_g = 0
    exptime_aggregate_r = 0
    exptime_aggregate_z = 0

    meansky_aggregate_g = 0
    meansky_aggregate_r = 0
    meansky_aggregate_z = 0

    galdepth_aggregate_g = 0
    galdepth_aggregate_r = 0
    galdepth_aggregate_z = 0

    psfdepth_aggregate_g = 0
    psfdepth_aggregate_r = 0
    psfdepth_aggregate_z = 0

    psfnorm_mean_aggregate_g = 0
    psfnorm_mean_aggregate_r = 0
    psfnorm_mean_aggregate_z = 0

    gausspsfdepth_aggregate_g = 0
    gausspsfdepth_aggregate_r = 0
    gausspsfdepth_aggregate_z = 0

    subpixels_covered = 0
    subpixels_covered_g = 0
    subpixels_covered_r = 0
    subpixels_covered_z = 0

    ccds_per_pixel = pixel2ccd_dict[sample_pixel]

    # Get values for singular systematics
    airmass_aggregate += airmass[ccds_per_pixel].sum() / len(ccds_per_pixel)

    # Get values for per band systematics

    mask_g = (filter_colour[ccds_per_pixel] == 'g')
    mask_r = (filter_colour[ccds_per_pixel] == 'r')
    mask_z = (filter_colour[ccds_per_pixel] == 'z')

    expt = exptime[ccds_per_pixel]
    exptime_g = expt[mask_g]
    exptime_r = expt[mask_r]
    exptime_z = expt[mask_z]

    see = seeing[ccds_per_pixel]
    seeing_g = see[mask_g]
    seeing_r = see[mask_r]
    seeing_z = see[mask_z]

    # Sky background
    sb = ccdskysb[ccds_per_pixel]
    ccdskysb_g = sb[mask_g]
    ccdskysb_r = sb[mask_r]
    ccdskysb_z = sb[mask_z]

    # Sky level
    msl = meansky[ccds_per_pixel]
    meansky_g = msl[mask_g]
    meansky_r = msl[mask_r]
    meansky_z = msl[mask_z]

    # Galaxy Depth
    gd = galdepth[ccds_per_pixel]
    galdepth_g = gd[mask_g]
    galdepth_r = gd[mask_r]
    galdepth_z = gd[mask_z]

    psfd = psfdepth[ccds_per_pixel]
    psfdepth_g = psfd[mask_g]
    psfdepth_r = psfd[mask_r]
    psfdepth_z = psfd[mask_z]

    psfnorm = psfnorm_mean[ccds_per_pixel]
    psfnorm_g = psfnorm[mask_g]
    psfnorm_r = psfnorm[mask_r]
    psfnorm_z = psfnorm[mask_z]

    gausspsfnorm = gausspsfdepth[ccds_per_pixel]
    gausspsf_g = gausspsfnorm[mask_g]
    gausspsf_r = gausspsfnorm[mask_r]
    gausspsf_z = gausspsfnorm[mask_z]

    g_exp = mask_g.sum()
    if g_exp > 0:
        exptime_aggregate_g = exptime_g.sum() / g_exp
        ccdskysb_aggregate_g = ccdskysb_g.sum() / g_exp
        meansky_aggregate_g = meansky_g.sum() / g_exp
        galdepth_aggregate_g = galdepth_g.sum() / g_exp
        seeing_aggregate_g = seeing_g.sum() / g_exp
        psfdepth_aggregate_g = psfdepth_g.sum() / g_exp
        psfnorm_mean_aggregate_g = psfnorm_g.sum() / g_exp
        gausspsfdepth_aggregate_g = gausspsf_g.sum() / g_exp

    r_exp = mask_r.sum()
    if r_exp > 0:
        subpixels_covered_r += 1
        exptime_aggregate_r = exptime_r.sum() / r_exp
        ccdskysb_aggregate_r = ccdskysb_r.sum() / r_exp
        meansky_aggregate_r = meansky_r.sum() / r_exp
        galdepth_aggregate_r = galdepth_r.sum() / r_exp
        seeing_aggregate_r = seeing_r.sum() / r_exp
        psfdepth_aggregate_r = psfdepth_r.sum() / r_exp
        psfnorm_mean_aggregate_r = psfnorm_r.sum() / r_exp
        gausspsfdepth_aggregate_r = gausspsf_r.sum() / r_exp

    z_exp = mask_z.sum()
    if z_exp > 0:
        subpixels_covered_z += 1
        exptime_aggregate_z = exptime_z.sum() / z_exp
        ccdskysb_aggregate_z = ccdskysb_z.sum() / z_exp
        meansky_aggregate_z = meansky_z.sum() / z_exp
        galdepth_aggregate_z = galdepth_z.sum() / z_exp
        seeing_aggregate_z = seeing_z.sum() / z_exp
        psfdepth_aggregate_z = psfdepth_z.sum() / z_exp
        psfnorm_mean_aggregate_z = psfnorm_z.sum() / z_exp
        gausspsfdepth_aggregate_z = gausspsf_z.sum() / z_exp

    if i == 280000:
        #print(int(i / 280000), '%')
        break


    systematics_per_pixel = [airmass_aggregate, ccdskysb_aggregate_g, ccdskysb_aggregate_r, ccdskysb_aggregate_z,
                             exptime_aggregate_g, exptime_aggregate_r, exptime_aggregate_z, meansky_aggregate_g,
                             meansky_aggregate_r, meansky_aggregate_z, galdepth_aggregate_g, galdepth_aggregate_r,
                             galdepth_aggregate_z, seeing_aggregate_g, seeing_aggregate_r, seeing_aggregate_z,
                             psfdepth_aggregate_g, psfdepth_aggregate_r, psfdepth_aggregate_z, psfnorm_mean_aggregate_g,
                             psfnorm_mean_aggregate_r, psfnorm_mean_aggregate_z, gausspsfdepth_aggregate_g,
                             gausspsfdepth_aggregate_r, gausspsfdepth_aggregate_z]

    pixel2systematics_dict[sample_pixel] = systematics_per_pixel

time_end = time.time()
time_passed = time_end - time_start
print()
print(f"{time_passed / 60:.5} minutes ({time_passed:.3} seconds) taken to classify 1%")
print()



1.0843 minutes (65.1 seconds) taken to classify 1%



In [ ]:
print(len(pixel2systematics_dict.keys()))

In [13]:
sys = pixel2systematics_dict[42053454]
print(sys)

[1.1846665700276693, 23.551791381835937, 22.642366027832033, 20.115696716308594, 90.0, 90.0, 90.0, 0.10299242734909057, 406842035.2, 2.454113578796387, 23.834757995605468, 23.309834289550782, 22.371339416503908, 1.286847972869873, 1.1721257209777831, 0.979978370666504, 24.080577087402343, 23.611331176757812, 22.77979736328125, 0.11759753227233886, 0.13574572801589965, 0.16844091415405274, 24.291725158691406, 23.7185302734375, 22.850808715820314]
[1.252696018592984, 22.608225176411292, 22.097897260616985, 20.290011485153656, 170.61290322580646, 146.92307692307693, 196.7109634551495, 712633918.7612903, 1414753043.6923077, 3370521487.734219, 23.139390120967743, 22.967117700821316, 22.26528433866279, 1.4669640325730846, 1.3650015806540465, 1.2865067288725083, 23.33409620715726, 23.200880784254807, 22.544498481623755, 0.10166181133639428, 0.11360806685227615, 0.12573954433301754, 23.55154989919355, 23.380731044671474, 22.688496028862126]


In [ ]:
with open(f'../../../bricks_data/2048_pixel2systematics_geometric_non_inclusive.pickle', 'wb') as f:
    pickle.dump(pixel2systematics_dict, f)
    f.close()